In [ ]:
import matplotlib
import networkx as nx
import pandas as pd

## 1. Load up the directed multigraph. Make sure the node names are strings.

In [ ]:
def load_multidigraph(path):
    """ Load a Directed Multigraph from a text file.
    
    Parameters
    ----------
        path: Path to text file containing data.
    
    Returns
    -------
        multidigraph: Networkx Directed Multigraph Object.
    """
    
    pass

## 2. How many unique sources did send an email? And how many emails were sent?

In [ ]:
def n_unique_email_sources(dataframe):
    """ Compute the number of unique email sources within a data set.
    
    Parameters
    ----------
        dataframe: Pandas Dataframe object with email data.
    
    Returns
    -------
        n: Integer number representing number of unique email sources.
    """
    
    pass

## 3. Assume that information to and from the research institute can only be exchanged through emails, When an email is sent to someone inside the institute, a communication channel is created, allowing the sender to provide information to the receiver, but not vice versa. Based on the emails sent on the data, is it possible for information to go from every sender to every other receiver?

In [ ]:
def fully_connected_one_way(digraph):
    """ Determine whether directed graph is fully connected.
    
    Parameters
    ----------
        digraph: Networkx Directed Graph representing email network.
    
    Returns
    -------
        fully_connected: Boolean result of network connection status.
            True: Network is fully connected.
            False: Network is not fully connected.
    """
    
    pass

## 4. Now assume that a communication channel established by an email allows information to be exchanged both ways. Based on the emails sent on the data, is it possible for information to go from every sender to every other receiver?

In [ ]:
def fully_connected_two_way(graph):
    """ Determine whether undirected graph is fully connected.
    
    Parameters
    ----------
        graph: Networkx Graph representing email network.
    
    Returns
    -------
        fully_connected: Boolean result of network connection status.
            True: Network is fully connected.
            False: Network is not fully connected.
    """
    
    pass

## 5. How many nodes are in the largest (in terms of nodes) weakly connected component?

In [ ]:
def size_largest_weak_component(digraph):
    """ Return the size of the largest weakly connected component in a graph.
    
    Parameters
    ----------
        digraph: Networkx Directed Graph.
    
    Returns
    -------
        n: Integer size of the largest weakly connected component.
    """
    
    pass

## 6. How many nodes are in the largest (in terms of nodes) strongly connected component?

In [ ]:
def size_largest_strong_component(digraph):
    """ Return the size of the largest strongly connected component in a graph.
    
    Parameters
    ----------
        digraph: Networkx Directed Graph.
    
    Returns
    -------
        n: Integer size of the largest strongly connected component.
    """
    
    pass

## 7. Using the function subgraph, find the subgraph of nodes in a largest strongly connected component. Call this graph G_sc. How many nodes are in this graph? 

## 8. What is the average distance between nodes in G_sc?

In [ ]:
def average_distance(graph):
    """ Compute the average distance between the nodes in a graph.
    
    Parameters
    ----------
        graph: Networkx Graph
    
    Returns
    -------
        d: Average distance between the nodes of `graph`.
    """
    
    pass

## 9. What is the largest possible distance between two employees in G_sc?

In [ ]:
def largest_distance(graph):
    """ Compute the largest distance between any two nodes in a graph.
    
    Parameters
    ----------
        graph: Networkx Graph.
    
    Returns
    -------
        d: Largest possible distance between two nodes in `graph`.
    """
    
    pass

## 10. What is the set of nodes in G_sc with eccentricity equal to the diameter?

In [ ]:
def set_w_eccentricity_equal_to_diameter(graph):
    """ Find nodes within a graph which have eccentricity equal to diameter.
    
    Parameters
    ----------
        graph: Networkx Graph.
    
    Returns
    -------
        target_set: Set of nodes with eccentricity equal to diameter.
    """
    
    pass

## 11. What is the set of node(s) in G_sc with eccentricity equal to the radius? How many nodes are connected to this node?

In [ ]:
def set_w_eccentricity_equal_to_diameter(graph):
    """ Find nodes within a graph which have eccentricity equal to diameter.
    
    Parameters
    ----------
        graph: Networkx Graph.
    
    Returns
    -------
        target_set: Set of nodes with eccentricity equal to diameter.
    """
    
    pass

## 12. Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc? How many nodes are connected to this node?

In [ ]:
def most_connected_w_path_equal_to_diameter(graph):
    """ Find the node with the most connections of length equal to diameter of the graph.
    
    Parameters
    ----------
        graph: Networkx Graph.
    
    Returns
    -------
        node: Node with the most connections, satisfying conditions.
        n: Number of nodes connected to `node`. 
    """
    
    pass

## 13. Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)?

In [ ]:
def prevent_comm_from_node_to_center(node, graph):
    """ Compute the number of nodes required to prevent communication between a node and the center of a graph.
    
    Parameters
    ----------
        node: Node to prevent from communicating with center of graph.
        graph: Graph containing `node`.
    
    Returns
    -------
        n: Number of nodes which need to be removed to prevent communications between `node` and the center of `graph`.
    """
    
    pass

## 14. Construct an undirected graph G_un using G_sc (you can ignore the attributes).

In [ ]:
def make_undirected(digraph):
    """ Convert Directed Graph into an Undirected Graph.
    
    Parameters
    ----------
        digraph: Networkx Directed Graph to convert.
    
    Returns
    -------
        graph: Networkx Undirected Graph representation of input `digraph`.
    """
    
    pass

## 15. What is the transitivity and average clustering coefficient of graph G_un?